### Load Data

In [1]:
import pandas as pd

#### Read datasets from all waves

In [ ]:
data_anchor = {}
for i in [1,2,3,4,5,6,7,8,9,10,11]:
    data_anchor["wave"+str(i)] = pd.read_stata('../../data/pairfam_v11/Data/Stata/anchor'+str(i)+'.dta')
    print('anchor dataset of wave '+str(i)+' was loaded successfully')

anchor dataset of wave 1 was loaded successfully
anchor dataset of wave 2 was loaded successfully
anchor dataset of wave 3 was loaded successfully
anchor dataset of wave 4 was loaded successfully
anchor dataset of wave 5 was loaded successfully


#### Create subsets with variables of interest

In [3]:
data_anchor_subset = {}

for i in range(1,12):
    
    subset = None
    
    if i not in [2,3,6]:
        subset = data_anchor['wave'+str(i)].loc[:, ['id', 'sat6', 'per1i6', 'age']]
        per1i6_codes = subset['per1i6'].cat.codes
        subset['per1i6'] = per1i6_codes - max(per1i6_codes) + 5
    
    else:
        subset = data_anchor['wave'+str(i)].loc[:, ['id', 'sat6']]

    # Recode values
    sat6_codes = subset['sat6'].cat.codes
    subset['sat6'] = sat6_codes - max(sat6_codes) + 10

    # Store subset
    data_anchor_subset["wave"+str(i)] = subset

#### Convert to Wide Format

In [4]:
for i in [1,4,5,7,8,9,10,11]:
    
    # Prepare column names
    df_temp = data_anchor_subset["wave"+str(i)].copy()
    df_temp.rename(columns={'sat6': 'sat6_T'+str(i), 'per1i6': 'per1i6_T'+str(i)}, inplace=True)
    df_temp.drop(['age'], axis=1, inplace=True)
    
    # Join subsets on anchor id
    if i==1:
        data_anchor_subset['join'] = df_temp
    else:
        data_anchor_subset['join'] = pd.merge(data_anchor_subset['join'], df_temp, on=['id']) 
    
# Remove rows with NANs
data_anchor_subset['join'] = data_anchor_subset['join'][(data_anchor_subset['join'] >= 0).all(1)]

# Rename columns
data_anchor_subset['join'] = data_anchor_subset['join'].rename(columns={'sat6_T1':  'sat6_T0', 'per1i6_T1':  'per1i6_T0',
                                                                            'sat6_T4':  'sat6_T1', 'per1i6_T4':  'per1i6_T1',
                                                                            'sat6_T5':  'sat6_T2', 'per1i6_T5':  'per1i6_T2',
                                                                            'sat6_T7':  'sat6_T3', 'per1i6_T7':  'per1i6_T3',
                                                                            'sat6_T8':  'sat6_T4', 'per1i6_T8':  'per1i6_T4',
                                                                            'sat6_T9':  'sat6_T5', 'per1i6_T9':  'per1i6_T5',
                                                                            'sat6_T10': 'sat6_T6', 'per1i6_T10': 'per1i6_T6',
                                                                            'sat6_T11': 'sat6_T7', 'per1i6_T11': 'per1i6_T7'})

# Add dT columns
data_anchor_subset['join']['dT1'] = 3
data_anchor_subset['join']['dT2'] = 1
data_anchor_subset['join']['dT3'] = 2
data_anchor_subset['join']['dT4'] = 1
data_anchor_subset['join']['dT5'] = 1
data_anchor_subset['join']['dT6'] = 1
data_anchor_subset['join']['dT7'] = 1

# Drop columns
data_anchor_subset['join'] = data_anchor_subset['join'].drop(['id'], axis=1)

# Export
data_anchor_subset['join'][0:1000].to_csv('../../data/samples/old_sample.csv', index=False)

#### Convert to Long Format

In [9]:
# Concat subsets
data_anchor_subset['union'] = pd.concat([data_anchor_subset['wave1'], 
                                         data_anchor_subset['wave2'],
                                         data_anchor_subset['wave3'],
                                         data_anchor_subset['wave4'],
                                         data_anchor_subset['wave5'],
                                         data_anchor_subset['wave6'],
                                         data_anchor_subset['wave7'],
                                         data_anchor_subset['wave8'],
                                         data_anchor_subset['wave9'],
                                         data_anchor_subset['wave10'],
                                         data_anchor_subset['wave11']])

# Remove rows with NANs
data_anchor_subset['union'] = data_anchor_subset['union'][(data_anchor_subset['union'] >= 0).all(1)]

# Recode IDs
# data_anchor_subset['union']['id'] = data_anchor_subset['union']['id'].astype('category').cat.codes

# Export
data_anchor_subset['union'].to_csv('../../data/samples/sample_all_complete.csv', index=False)